In [1]:
# Author: Hassan Ismail Fawaz <hassan.ismail-fawaz@uha.fr>
#         Germain Forestier <germain.forestier@uha.fr>
#         Jonathan Weber <jonathan.weber@uha.fr>
#         Lhassane Idoumghar <lhassane.idoumghar@uha.fr>
#         Pierre-Alain Muller <pierre-alain.muller@uha.fr>
# License: GPL3

import numpy as np
import pandas as pd
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'

import operator
import math
from scipy.stats import wilcoxon
from scipy.stats import friedmanchisquare
import networkx

# inspired from orange3 https://docs.orange.biolab.si/3/data-mining-library/reference/evaluation.cd.html
def graph_ranks(avranks, names, p_values, cd=None, cdmethod=None, lowv=None, highv=None,
                width=6, textspace=1, reverse=False, filename=None, labels=False, **kwargs):
    """
    Draws a CD graph, which is used to display  the differences in methods'
    performance. See Janez Demsar, Statistical Comparisons of Classifiers over
    Multiple Data Sets, 7(Jan):1--30, 2006.

    Needs matplotlib to work.

    The image is ploted on `plt` imported using
    `import matplotlib.pyplot as plt`.

    Args:
        avranks (list of float): average ranks of methods.
        names (list of str): names of methods.
        cd (float): Critical difference used for statistically significance of
            difference between methods.
        cdmethod (int, optional): the method that is compared with other methods
            If omitted, show pairwise comparison of methods
        lowv (int, optional): the lowest shown rank
        highv (int, optional): the highest shown rank
        width (int, optional): default width in inches (default: 6)
        textspace (int, optional): space on figure sides (in inches) for the
            method names (default: 1)
        reverse (bool, optional):  if set to `True`, the lowest rank is on the
            right (default: `False`)
        filename (str, optional): output file name (with extension). If not
            given, the function does not write a file.
        labels (bool, optional): if set to `True`, the calculated avg rank
        values will be displayed
    """
    try:
        import matplotlib
        import matplotlib.pyplot as plt
        from matplotlib.backends.backend_agg import FigureCanvasAgg
    except ImportError:
        raise ImportError("Function graph_ranks requires matplotlib.")

    width = float(width)
    textspace = float(textspace)

    def nth(l, n):
        """
        Returns only nth elemnt in a list.
        """
        n = lloc(l, n)
        return [a[n] for a in l]

    def lloc(l, n):
        """
        List location in list of list structure.
        Enable the use of negative locations:
        -1 is the last element, -2 second last...
        """
        if n < 0:
            return len(l[0]) + n
        else:
            return n

    def mxrange(lr):
        """
        Multiple xranges. Can be used to traverse matrices.
        This function is very slow due to unknown number of
        parameters.

        >>> mxrange([3,5])
        [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

        >>> mxrange([[3,5,1],[9,0,-3]])
        [(3, 9), (3, 6), (3, 3), (4, 9), (4, 6), (4, 3)]

        """
        if not len(lr):
            yield ()
        else:
            # it can work with single numbers
            index = lr[0]
            if isinstance(index, int):
                index = [index]
            for a in range(*index):
                for b in mxrange(lr[1:]):
                    yield tuple([a] + list(b))

    def print_figure(fig, *args, **kwargs):
        canvas = FigureCanvasAgg(fig)
        canvas.print_figure(*args, **kwargs)

    sums = avranks

    nnames = names
    ssums = sums

    if lowv is None:
        lowv = min(1, int(math.floor(min(ssums))))
    if highv is None:
        highv = max(len(avranks), int(math.ceil(max(ssums))))

    cline = 0.4

    k = len(sums)

    lines = None

    linesblank = 0
    scalewidth = width - 2 * textspace

    def rankpos(rank):
        if not reverse:
            a = rank - lowv
        else:
            a = highv - rank
        return textspace + scalewidth / (highv - lowv) * a

    distanceh = 0.25

    cline += distanceh

    # calculate height needed height of an image
    minnotsignificant = max(2 * 0.2, linesblank)
    height = cline + ((k + 1) / 2) * 0.2 + minnotsignificant

    fig = plt.figure(figsize=(width, height))
    fig.set_facecolor('white')
    ax = fig.add_axes([0, 0, 1, 1])  # reverse y axis
    ax.set_axis_off()

    hf = 1. / height  # height factor
    wf = 1. / width

    def hfl(l):
        return [a * hf for a in l]

    def wfl(l):
        return [a * wf for a in l]

    # Upper left corner is (0,0).
    ax.plot([0, 1], [0, 1], c="w")
    ax.set_xlim(0, 1)
    ax.set_ylim(1, 0)

    def line(l, color='k', **kwargs):
        """
        Input is a list of pairs of points.
        """
        ax.plot(wfl(nth(l, 0)), hfl(nth(l, 1)), color=color, **kwargs)

    def text(x, y, s, *args, **kwargs):
        ax.text(wf * x, hf * y, s, *args, **kwargs)

    line([(textspace, cline), (width - textspace, cline)], linewidth=2)

    bigtick = 0.3
    smalltick = 0.15
    linewidth = 2.0
    linewidth_sign = 4.0

    tick = None
    for a in list(np.arange(lowv, highv, 0.5)) + [highv]:
        tick = smalltick
        if a == int(a):
            tick = bigtick
        line([(rankpos(a), cline - tick / 2),
              (rankpos(a), cline)],
             linewidth=2)

    for a in range(lowv, highv + 1):
        text(rankpos(a), cline - tick / 2 - 0.05, str(a),
             ha="center", va="bottom", size=16)

    k = len(ssums)

    def filter_names(name):
        return name

    space_between_names = 0.24

    for i in range(math.ceil(k / 2)):
        chei = cline + minnotsignificant + i * space_between_names
        line([(rankpos(ssums[i]), cline),
              (rankpos(ssums[i]), chei),
              (textspace - 0.1, chei)],
             linewidth=linewidth)
        if labels:
            text(textspace + 0.3, chei - 0.075, format(ssums[i], '.4f'), ha="right", va="center", size=10)
        text(textspace - 0.2, chei, filter_names(nnames[i]), ha="right", va="center", size=16)

    for i in range(math.ceil(k / 2), k):
        chei = cline + minnotsignificant + (k - i - 1) * space_between_names
        line([(rankpos(ssums[i]), cline),
              (rankpos(ssums[i]), chei),
              (textspace + scalewidth + 0.1, chei)],
             linewidth=linewidth)
        if labels:
            text(textspace + scalewidth - 0.3, chei - 0.075, format(ssums[i], '.4f'), ha="left", va="center", size=10)
        text(textspace + scalewidth + 0.2, chei, filter_names(nnames[i]),
             ha="left", va="center", size=16)

    # no-significance lines
    def draw_lines(lines, side=0.05, height=0.1):
        start = cline + 0.2

        for l, r in lines:
            line([(rankpos(ssums[l]) - side, start),
                  (rankpos(ssums[r]) + side, start)],
                 linewidth=linewidth_sign)
            start += height
            print('drawing: ', l, r)

    # draw_lines(lines)
    start = cline + 0.2
    side = -0.02
    height = 0.1

    # draw no significant lines
    # get the cliques
    cliques = form_cliques(p_values, nnames)
    i = 1
    achieved_half = False
    print(nnames)
    for clq in cliques:
        if len(clq) == 1:
            continue
        print(clq)
        min_idx = np.array(clq).min()
        max_idx = np.array(clq).max()
        if min_idx >= len(nnames) / 2 and achieved_half == False:
            start = cline + 0.25
            achieved_half = True
        line([(rankpos(ssums[min_idx]) - side, start),
              (rankpos(ssums[max_idx]) + side, start)],
             linewidth=linewidth_sign)
        start += height


def form_cliques(p_values, nnames):
    """
    This method forms the cliques
    """
    # first form the numpy matrix data
    m = len(nnames)
    g_data = np.zeros((m, m), dtype=np.int64)
    for p in p_values:
        if p[3] == False:
            i = np.where(nnames == p[0])[0][0]
            j = np.where(nnames == p[1])[0][0]
            min_i = min(i, j)
            max_j = max(i, j)
            g_data[min_i, max_j] = 1

    g = networkx.Graph(g_data)
    return networkx.find_cliques(g)


def draw_cd_diagram(df_perf=None, alpha=0.05, title=None, labels=False):
    """
    Draws the critical difference diagram given the list of pairwise classifiers that are
    significant or not
    """
    p_values, average_ranks, _ = wilcoxon_holm(df_perf=df_perf, alpha=alpha)

    print(average_ranks)

    for p in p_values:
        print(p)


    graph_ranks(average_ranks.values, average_ranks.keys(), p_values,
                cd=None, reverse=True, width=9, textspace=1.5, labels=labels)

    font = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'normal',
        'size': 22,
        }
    if title:
        plt.title(title,fontdict=font, y=0.9, x=0.5)
    plt.savefig(f'cd-diagram-{title}.png',bbox_inches='tight')

def wilcoxon_holm(alpha=0.05, df_perf=None):
    """
    Applies the wilcoxon signed rank test between each pair of algorithm and then use Holm
    to reject the null's hypothesis
    """
    print(pd.unique(df_perf['Model']))
    # count the number of tested datasets per classifier
    df_counts = pd.DataFrame({'count': df_perf.groupby(
        ['Model']).size()}).reset_index()
    # get the maximum number of tested datasets
    print(df_counts)
    max_nb_datasets = df_counts['count'].max()
    # get the list of classifiers who have been tested on nb_max_datasets
    classifiers = list(df_counts.loc[df_counts['count'] == max_nb_datasets]
                       ['Model'])
    # test the null hypothesis using friedman before doing a post-hoc analysis
    friedman_p_value = friedmanchisquare(*(
        np.array(df_perf.loc[df_perf['Model'] == c]['Score'])
        for c in classifiers))[1]
    if friedman_p_value >= alpha:
        # then the null hypothesis over the entire classifiers cannot be rejected
        print('the null hypothesis over the entire classifiers cannot be rejected')
        exit()
    # get the number of classifiers
    m = len(classifiers)
    # init array that contains the p-values calculated by the Wilcoxon signed rank test
    p_values = []
    # loop through the algorithms to compare pairwise
    for i in range(m - 1):
        # get the name of classifier one
        classifier_1 = classifiers[i]
        # get the performance of classifier one
        perf_1 = np.array(df_perf.loc[df_perf['Model'] == classifier_1]['Score']
                          , dtype=np.float64)
        for j in range(i + 1, m):
            # get the name of the second classifier
            classifier_2 = classifiers[j]
            # get the performance of classifier one
            perf_2 = np.array(df_perf.loc[df_perf['Model'] == classifier_2]
                              ['Score'], dtype=np.float64)
            # calculate the p_value
            p_value = wilcoxon(perf_1, perf_2, zero_method='pratt')[1]
            # appen to the list
            p_values.append((classifier_1, classifier_2, p_value, False))
    # get the number of hypothesis
    k = len(p_values)
    # sort the list in acsending manner of p-value
    p_values.sort(key=operator.itemgetter(2))

    # loop through the hypothesis
    for i in range(k):
        # correct alpha with holm
        new_alpha = float(alpha / (k - i))
        # test if significant after holm's correction of alpha
        if p_values[i][2] <= new_alpha:
            p_values[i] = (p_values[i][0], p_values[i][1], p_values[i][2], True)
        else:
            # stop
            break
    # compute the average ranks to be returned (useful for drawing the cd diagram)
    # sort the dataframe of performances
    sorted_df_perf = df_perf.loc[df_perf['Model'].isin(classifiers)]. \
        sort_values(['Model', 'dataset_name'])
    # get the rank data
    rank_data = np.array(sorted_df_perf['Score']).reshape(m, max_nb_datasets)
    print(rank_data)
    # create the data frame containg the accuracies
    df_ranks = pd.DataFrame(data=rank_data, index=np.sort(classifiers), columns=
    np.unique(sorted_df_perf['dataset_name']))

    # number of wins
    dfff = df_ranks.rank(ascending=False)
    print(dfff[dfff == 1.0].sum(axis=1))

    # average the ranks
    average_ranks = df_ranks.rank(ascending=False).mean(axis=1).sort_values(ascending=False)
    # return the p-values and the average ranks
    return p_values, average_ranks, max_nb_datasets





In [18]:
df = pd.read_csv('benchmark_results_aggregated.csv', index_col=False).reset_index(drop=True)

accuracy_df = df[df["Metric"] == "accuracy"]
precision_df = df[df["Metric"] == "precision"]
recall_df = df[df["Metric"] == "recall"]
f1_df = df[df["Metric"] == "f1"]
balanced_error_df = df[df["Metric"] == "balanced_error"]
auc_df = df[df["Metric"] == "auc"]


In [19]:
#make 6 concats of np.arange(1500)
n_testing_datasets = 6
total_amount_of_time_series = 1000*3*4
concats = [np.arange(total_amount_of_time_series)]*n_testing_datasets
#concatenate them
concats = np.concatenate(concats)
concats

array([    0,     1,     2, ..., 11997, 11998, 11999])

In [20]:
balanced_error_df['Metric'] = concats
balanced_error_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)

accuracy_df['Metric'] = concats
accuracy_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)

precision_df['Metric'] = concats
precision_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)

recall_df['Metric'] = concats
recall_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)

f1_df['Metric'] = concats
f1_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)





/tmp/ipykernel_19350/3412831708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balanced_error_df['Metric'] = concats
/tmp/ipykernel_19350/3412831708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balanced_error_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)
/tmp/ipykernel_19350/3412831708.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [26]:
auc_df['Metric'] = np.concatenate([np.arange(11997)]*(n_testing_datasets-1))
auc_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)


/tmp/ipykernel_19350/370018895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auc_df['Metric'] = np.concatenate([np.arange(11997)]*(n_testing_datasets-1))
/tmp/ipykernel_19350/370018895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auc_df.rename(columns={'Metric': 'dataset_name'}, inplace=True)


In [21]:
balanced_error_df.dataset_name

4             0
10            1
16            2
22            3
28            4
          ...  
419959    11995
419965    11996
419971    11997
419977    11998
419983    11999
Name: dataset_name, Length: 72000, dtype: int64

In [22]:
balanced_error_df

,Model,dataset_name,Score
4,D2C,0,0.500000
10,D2C,1,0.421569
16,D2C,2,0.437500
22,D2C,3,0.490132
28,D2C,4,0.388889
...,...,...,...
419959,DYNOTEARS,11995,0.475000
419965,DYNOTEARS,11996,0.475000
419971,DYNOTEARS,11997,0.450000
419977,DYNOTEARS,11998,0.475000


In [29]:
names = ['accuracy', 'precision', 'recall', 'f1', 'balanced_error', 'auc']
for df_idx, df in enumerate([accuracy_df, precision_df, recall_df, f1_df, balanced_error_df, auc_df]):
    title = names[df_idx]
    draw_cd_diagram(df_perf=df, title=title, labels=True)


['D2C' 'Granger' 'PCMCI' 'VAR' 'VARLiNGAM' 'DYNOTEARS']
       Model  count
0        D2C  12000
1  DYNOTEARS  12000
2    Granger  12000
3      PCMCI  12000
4        VAR  12000
5  VARLiNGAM  12000


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

[[0.65       0.2        0.39285714 ... 0.60526316 0.675      0.55      ]
 [0.5        0.65       0.71428571 ... 0.60526316 0.6        0.575     ]
 [0.65       0.5        0.42857143 ... 0.47368421 0.6        0.5       ]
 [0.6        0.5        0.53571429 ... 0.42105263 0.425      0.55      ]
 [0.65       0.7        0.60714286 ... 0.5        0.4        0.525     ]
 [0.4        0.7        0.5        ... 0.60526316 0.4        0.5       ]]
D2C          3156.0
DYNOTEARS    2402.0
Granger       369.0
PCMCI         627.0
VAR           577.0
VARLiNGAM    1782.0
dtype: float64
Granger      4.963208
PCMCI        4.228833
VAR          3.262292
VARLiNGAM    3.143667
DYNOTEARS    2.728792
D2C          2.673208
dtype: float64
('D2C', 'Granger', 0.0, True)
('D2C', 'PCMCI', 0.0, True)
('DYNOTEARS', 'Granger', 0.0, True)
('DYNOTEARS', 'PCMCI', 0.0, True)
('Granger', 'PCMCI', 0.0, True)
('Granger', 'VAR', 0.0, True)
('Granger', 'VARLiNGAM', 0.0, True)
('PCMCI', 'VAR', 0.0, True)
('PCMCI', 'VARLiNGAM', 0.

findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial


['D2C' 'Granger' 'PCMCI' 'VAR' 'VARLiNGAM' 'DYNOTEARS']
       Model  count
0        D2C  12000
1  DYNOTEARS  12000
2    Granger  12000
3      PCMCI  12000
4        VAR  12000
5  VARLiNGAM  12000


...

[[0.5        0.81318681 0.5        ... 0.39473684 0.325      0.45      ]
 [0.61538462 0.4010989  0.5        ... 0.39473684 0.4        0.425     ]
 [0.4010989  0.48351648 0.5875     ... 0.52631579 0.4        0.5       ]
 [0.30769231 0.45054945 0.4        ... 0.57894737 0.575      0.45      ]
 [0.46703297 0.42857143 0.575      ... 0.5        0.6        0.475     ]
 [0.59340659 0.32967033 0.5        ... 0.39473684 0.6        0.5       ]]
D2C           853.0
DYNOTEARS     838.0
Granger      3294.0
PCMCI        3546.0
VAR          1309.0
VARLiNGAM     809.0
dtype: float64
VARLiNGAM    4.513042
D2C          4.401125
DYNOTEARS    3.441042
VAR          3.041042
Granger      2.909958
PCMCI        2.693792
dtype: float64
('D2C', 'DYNOTEARS', 0.0, True)
('D2C', 'Granger', 0.0, True)
('D2C', 'PCMCI', 0.0, True)
('D2C', 'VAR', 0.0, True)
('DYNOTEARS', 'VARLiNGAM', 0.0, True)
('Granger', 'VARLiNGAM', 0.0, True)
('PCMCI', 'VARLiNGAM', 0.0, True)
('VAR', 'VARLiNGAM', 0.0, True)
('DYNOTEARS', 'PCMCI', 

findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial


['D2C' 'PCMCI' 'VAR' 'VARLiNGAM' 'DYNOTEARS']
       Model  count
0        D2C  11997
1  DYNOTEARS  11997
2      PCMCI  11997
3        VAR  11997
4  VARLiNGAM  11997


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

[[0.78021978 0.67320261 0.81666667 ... 0.4825     0.77125    0.58033241]
 [0.5125     0.45       0.64285714 ... 0.525      0.625      0.575     ]
 [0.66666667 0.74722222 0.60555556 ... 0.6        0.75       0.28214286]
 [0.85620915 1.         0.99346405 ... 0.97802198 0.85620915 0.96732026]
 [0.825      0.85       0.74444444 ... 0.68       0.82857143 0.66666667]]
D2C          2944.0
DYNOTEARS     986.0
PCMCI         731.0
VAR          3454.0
VARLiNGAM    2686.0
dtype: float64
PCMCI        3.888305
DYNOTEARS    3.570684
D2C          2.558806
VARLiNGAM    2.528174
VAR          2.454030
dtype: float64
('D2C', 'DYNOTEARS', 0.0, True)
('D2C', 'PCMCI', 0.0, True)
('DYNOTEARS', 'VAR', 0.0, True)
('DYNOTEARS', 'VARLiNGAM', 0.0, True)
('PCMCI', 'VAR', 0.0, True)
('PCMCI', 'VARLiNGAM', 0.0, True)
('DYNOTEARS', 'PCMCI', 3.30118439023889e-156, True)
('VAR', 'VARLiNGAM', 1.0566812493629572e-60, True)
('D2C', 'VAR', 9.288322219826486e-16, True)
('D2C', 'VARLiNGAM', 0.11051433123844845, False)
Index(